In [ ]:
import numpy as np
import pandas as pd
import os
import csv
import datetime
import json
import math

In [ ]:
path = '''/w/225/1/chess/tactics/'''
os.chdir(path)
#filename = '''user_tactics_problem.csv_00'''

In [ ]:
!ls

In [ ]:
class Player:
    def __init__(self,rating=1720,rd=350):
        self.rating = rating
        self.rd = rd
        self.p = 3 * (math.log(10)**2) / ((math.pi ** 2) * 400**2)
        self.q = math.log(10)/400
        self.c = 900
    
    def getRating(self):
        return self.rating
    
    def getRd(self):
        return self.rd
    
    def f(self, oppRd):
        return 1/math.sqrt(1 + (self.p * (oppRd ** 2)))
    
    def E(self, oppRating, oppRd):
        return 1/(1 + 10**(-1 * (self.rating - oppRating) * self.f(oppRd) / 400))
    
    def K(self, oppRating, oppRd):
        k = self.q * self.f(oppRd)/ ((1/self.rd ** 2) + (self.q **2)*(self.f(oppRd)**2)*self.E(oppRating, oppRd) * (1-self.E(oppRating, oppRd)))
        if k < 16:
            return 16
        else:
            return k
    
    def update_no_games(self, timeWithoutPlaying):
        #print(timeWithoutPlaying)
        self.rd = math.sqrt(self.rd**2 + self.c * timeWithoutPlaying)
        if self.rd > 350:
            self.rd = 350
    
    def update_player(self, oppRating, oppRd, result):
        self.rating = self.rating + (self.K(oppRating, oppRd) * (int(result) - self.E(oppRating, oppRd)))
        self.rd = (math.sqrt((1/self.rd ** 2) + (self.q ** 2 ) * (self.f(oppRd) ** 2) * self.E(oppRating, oppRd) * (1-self.E(oppRating, oppRd))))** -1 

In [ ]:
#df = pd.read_csv(filename, nrows = 10)
#print(df)
'''

userDict = {} #user_ID:Rating
problemDict = {} # ID:Rating
userGamesPlayed = {} #user_ID:number of games played
userLastPlayed = {} #user_ID:date of last game played

def getRating(user, Dict, default):
    if user not in Dict:
        Dict[user] = default
    return Dict[user]    
'''
def getRating(user, Dict):
    if user not in Dict:
        Dict[user] = Player()
    return Dict[user].getRating()

def getDateTime(raw_dt):
    a, b, c = [int(i) for i in raw_dt.split()[0].split("-")]
    return datetime.date(a, b, c)

#with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userGamesPlayed.json') as json_file:
#    userGamesPlayed = json.load(json_file)
with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userLastPlayed.json') as json_file:
    userLastPlayed = json.load(json_file)
for key in userLastPlayed.keys():
    userLastPlayed[key] = getDateTime(userLastPlayed[key])
with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userdict.json') as json_file:
    user_stats = json.load(json_file)
with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/problemdict.json') as json_file:
    problem_stats = json.load(json_file)

In [ ]:
userDict = {}
for key in user_stats.keys():
    stat = user_stats[key]
    userDict[key] = Player(stat['rating'],stat['rd'])

In [ ]:
problemDict = {}
for key in problem_stats.keys():
    stat = problem_stats[key]
    problemDict[key] = Player(stat['rating'],stat['rd'])

In [ ]:
import sys
sys.stdout = open('/dev/stdout', 'w')

In [ ]:
#save ints, not floats
#write to file every 100000 lines or so to speed up
#write own 

chunksUsed = 0
newCSV = open("glicko_user_tactics_problem_all.csv", "a", newline= "")
newCSV = csv.writer(newCSV)
#newCSV.writerow(["user_tactics_problem_id", "user_hash", "create_date", "date", "last_game_date", "seconds", "is_passed", "correct_move_count", "rating_change", "tactics_problem_id", "ratingUser", "varianceUser", "ratingProblem", "varianceProblem", "userGamesPlayed"])
problemProblems = []
for i in range(1,10):
    filename = 'user_tactics_problem.csv_0{}.bz2'.format(i)
    print('Trying file: {}'.format(filename))
    for chunk in pd.read_csv(filename, names = ['user_tactics_problem_id','user_hash','create_date','seconds','is_passed','correct_move_count','rating_change','tactics_problem_id'],chunksize=10000): 
        rows = []
        for index, row in chunk.iterrows():
            #print(row.user_hash)
            user = row["user_hash"]
            problem = row["tactics_problem_id"]
            if str(problem) in problemProblems:
                continue
            winner = row["is_passed"]
            date = getDateTime(row["create_date"])
            if user not in userLastPlayed:
                userLastPlayed[user] = date
            userRating = getRating(user, userDict)
            daysMissed = (date-userLastPlayed[user]).days
            userDict[user].update_no_games(daysMissed)
            lastGame = userLastPlayed[user]
            userLastPlayed[user] = date
            userRd = userDict[user].getRd()
            problemRating = getRating(problem, problemDict)
            problemRd = problemDict[problem].getRd()
            #userDict[user], problemDict[problem] = EloRating(userRating, problemRating, winner)
            try:
                userDict[user].update_player(problemRating, problemRd, bool(winner))
                problemDict[problem].update_player(userRating, userRd, not bool(winner))
                if user not in userGamesPlayed:
                    userGamesPlayed[user] = 1
                else:
                    userGamesPlayed[user] = userGamesPlayed[user] + 1
                newRow = [row["user_tactics_problem_id"], row["user_hash"], row["create_date"], str(date), str(lastGame), row["seconds"], row["is_passed"], row["correct_move_count"], row["rating_change"], row["tactics_problem_id"], str(int(userRating)), str(int(userRd)), str(int(problemRating)), str(int(problemRd)), str(userGamesPlayed[user]-1)]
                rows.append(newRow)
            except:
                print("error with rating,  user:", user)
                print("problem ID:", problem)
                problemProblems.append(str(problem))

        newCSV.writerows(rows)    
        chunksUsed += 1
        if chunksUsed % 10 == 0:
            print('Chunks used: {}'.format(chunksUsed))
        #break
    #break
    print('Done with file: {}'.format(i))

<h2>Save JSON file</h2>

In [ ]:
import json
from tqdm import tqdm_notebook as tqdm
userDictSave = {}
for has in tqdm(userDict.keys()):
    player = userDict[has]
    dic = {'rating':player.getRating(),'rd':player.getRd()}
    userDictSave[has] = dic
    #print(player.getRating(),player.getRd())
    
with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userdict.json', 'w') as outfile:
    json.dump(userDictSave, outfile)
    
problemDictSave = {}
for has in tqdm(problemDict.keys()):
    player = problemDict[has]
    dic = {'rating':player.getRating(),'rd':player.getRd()}
    problemDictSave[has] = dic
    
with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/problemdict.json', 'w') as outfile:
    json.dump(problemDictSave, outfile)
    


In [ ]:
#with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userGamesPlayed.json', 'w') as outfile:
#    json.dump(userGamesPlayed,outfile)
#with open('/h/224/evanb/chess_dim_skill/user_puzzle_dics/userLastPlayed.json', 'w') as outfile:
#    json.dump(userLastPlayed, outfile,default=str)